### Stitching up the video

In [1]:
import cv2 as cv
import glob
import os


img_files = sorted(glob.glob('YCBvideo/data/0001/*.jpg'))
sample_img = cv.imread(img_files[0])
fourcc = cv.VideoWriter_fourcc(*'mp4v')
h,w,l = sample_img.shape
video = cv.VideoWriter('sample1.mp4',fourcc,1,(w,h),True)
for img in img_files:
    i= cv.imread(img)
    video.write(i)
    
video.release()

### playing with corner detection


In [ ]:
import numpy as np
import cv as cv
import pandas as pd


# Read the image and convert to grayscale
frame = cv.imread('YCBvideo/data/0001/000001-color.jpg')
frame2 = cv.imread('YCBvideo/data/0001/000001-color.jpg')

bounding_box_df = pd.read_csv('YCBvideo/data/0001/000001-box.txt',sep=' ',header=None)
df = bounding_box_df.iloc[0]
padding = 0
x_min ,x_max ,y_min, y_max = int(df[1])+padding,int(df[2])+padding,int(df[3])+padding,int(df[4]+padding)
x_min ,x_max ,y_min, y_max

cv.rectangle(frame2,(x_min,y_min),(x_max,y_max),(0,255,0),2)
cv.imshow('im',frame2)
cv.waitKey(0)
cv.destroyAllWindows()

In [12]:
import cv2 as cv
frame = cv.imread('YCBvideo/data/0001/000001-label.png')
print(np.min(frame),np.max(frame),np.unique(frame),frame.shape)
o1 = np.zeros_like(frame)
o1[frame==1] = 255
cv.imshow('mask',o1)
cv.waitKey(0)
cv.destroyAllWindows()

0 5 [0 1 2 3 5] (480, 640, 3)


In [ ]:

gray_frame = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

mask = np.zeros_like(gray_frame)
mask[y_min:y_max,x_min:x_max] = 255


feature_params = dict(maxCorners=20, qualityLevel=0.5, minDistance=8, blockSize=10)
corners = cv.goodFeaturesToTrack(gray_frame, mask=mask ,**feature_params)
for i in corners:
    x, y = i.ravel()
    cv.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)

# Display the image with corners marked
cv.imshow('Corners', frame)
cv.waitKey(0)
cv.destroyAllWindows()


### working code

In [18]:
import cv2
import numpy as np
import pandas as pd


# === INPUT ===
video_path = "sample1.mp4"
bounding_box_df = pd.read_csv('YCBvideo/data/0001/000001-box.txt',sep=' ',header=None)
label_frame = cv2.imread('YCBvideo/data/0001/000001-label.png',cv.IMREAD_GRAYSCALE)
df = bounding_box_df.iloc[0]
padding = 0
x_min ,x_max ,y_min, y_max = int(df[1])+padding,int(df[2])+padding,int(df[3])+padding,int(df[4]+padding)
x_min ,x_max ,y_min, y_max

# === PARAMETERS ===
feature_params = dict(maxCorners=200, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=5,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# === INITIALIZATION ===
cap = cv2.VideoCapture(video_path)
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Mask for features only inside the initial bounding box
mask = np.zeros_like(old_gray)
print(mask.shape,label_frame.shape)
mask[label_frame==2] = 255
print(np.unique(mask))
# Detect initial features to track
p0 = cv2.goodFeaturesToTrack(old_gray, mask=mask, **feature_params)

# Create image for drawing optical flow tracks
mask_lines = np.zeros_like(old_frame)
pu= True
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    
    # Select good points
    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        # Draw the tracks
        # if pu:
        #     pu= False
        #     print(p1,st,good_new,sep='\n \n')
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            if pu:
                pu= False
                print(p1,st,good_new,new,sep='Here \n \n')
            a, b = new.ravel()
            c, d = old.ravel()
            # cv2.line(mask_lines, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
            cv2.circle(frame, (int(a), int(b)), 3, (0, 0, 255), -1)

        # if len(good_new) > 0:
        #     # Get all x and y coordinates of tracked points
        #     x_coords = good_new[:, 0]
        #     y_coords = good_new[:, 1]
        #     x_min,y_min,x_max,y_max = np.min(x_coords),np.min(y_coords),np.max(x_coords),np.max(y_coords)
        #     h,w = y_max-y_min,x_max-x_min
        #     padding =50
        #     # Compute new bounding box
        #     new_x_min = int(x_min -w)
        #     new_x_max = int(x_max +w)
        #     new_y_min = int(y_min -h)
        #     new_y_max = int(y_max + h)
            
        #     # Draw updated bounding box
        #     cv2.rectangle(frame, (new_x_min, new_y_min), 
        #                 (new_x_max, new_y_max), (255, 0, 0), 2)
        
        frame_count = 10
        print(p0.shape,mask.shape)
        warped_mask = cv2.remap(
        mask, 
        p1.reshape(-1, 2).astype(np.float16),  # New points from optical flow
        None, 
        interpolation=cv2.INTER_LINEAR
        )
        # Threshold to maintain binary mask
        warped_mask = (warped_mask > 127).astype(np.uint8) * 255
        # After optical flow computation:
        if frame_count % 10 == 0:  # Re-initialize every 10 frames
            p0 = cv2.goodFeaturesToTrack(frame_gray, mask=warped_mask, **feature_params)
        
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)


        # Update previous frame and points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    # Overlay lines on the current frame
    output = cv2.add(frame, mask_lines)
    cv2.imshow('Optical Flow Tracking', output)

    if cv2.waitKey(27) & 0xFF == ord('q'):  # Press ESC to break
        break

cap.release()
cv2.destroyAllWindows()


(480, 640) (480, 640)
[  0 255]
[[[293.25668 140.8232 ]]

 [[270.34924 143.85898]]

 [[255.42058 153.94127]]

 [[164.41548 178.97842]]

 [[284.3657  230.99585]]

 [[258.3537  145.85437]]

 [[247.33624 147.91739]]

 [[204.41205 195.91687]]

 [[278.43625 179.90427]]

 [[255.3555  197.93575]]

 [[215.43845 237.94972]]

 [[269.39868 184.83138]]

 [[270.4101  191.85257]]

 [[275.31274 138.87344]]

 [[296.33286 188.87415]]

 [[211.23651 172.92563]]

 [[276.42566 211.93138]]

 [[284.30203 141.84187]]

 [[301.378   201.91818]]

 [[312.3365  148.87537]]

 [[233.21907 161.94226]]

 [[255.34962 190.94832]]

 [[309.32968 185.88972]]

 [[200.48656 182.8677 ]]

 [[324.3093  155.81769]]

 [[294.41434 205.88663]]

 [[219.38225 193.88255]]

 [[283.41763 211.89986]]

 [[275.382   201.93155]]

 [[309.29684 302.96213]]

 [[246.36252 218.97653]]

 [[309.38214 194.89478]]

 [[277.4676  219.90648]]

 [[282.3381  238.0033 ]]]Here 
 
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/imgwarp.cpp:1911: error: (-215:Assertion failed) ((map1.type() == CV_32FC2 || map1.type() == CV_16SC2) && map2.empty()) || (map1.type() == CV_32FC1 && map2.type() == CV_32FC1) in function 'remap'


### new code

In [ ]:
import cv2
import pandas as pd
import numpy as np


# Initialization (keep existing code)
bounding_box_df = pd.read_csv('YCBvideo/data/0001/000001-box.txt', sep=' ', header=None)
df = bounding_box_df.iloc[0]
padding = 0  # Add padding for better feature coverage
x_min, x_max, y_min, y_max = int(df[1])+padding, int(df[2])+padding, int(df[3])+padding, int(df[4])+padding


cap = cv2.VideoCapture('sample1.mp4')
ret, frame = cap.read()
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Initialize mask with expanded ROI
mask = np.zeros_like(old_gray)
mask[y_min:y_max, x_min:x_max] = 255

feature_params = dict(
    maxCorners=1000,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7
)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=mask, **feature_params)

lk_params = dict(
    winSize=(25, 25),
    maxLevel=3,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.03)
)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        
        if len(good_new) > 4:
            # Create convex hull mask
            points = good_new.reshape(-1, 2).astype(np.int32)
            hull = cv2.convexHull(points)
            
            # Create tracking visualization
            mask_vis = np.zeros_like(frame)
            cv2.fillPoly(mask_vis, [hull], (0, 255, 0))
            
            # Blend with original frame
            output = cv2.addWeighted(frame, 0.7, mask_vis, 0.3, 0)
            
            # Update previous points
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)
            
            # Optional: Redetect features periodically
            if len(good_new) < 20:  # Threshold for feature refresh
                p0 = cv2.goodFeaturesToTrack(old_gray, mask=mask_vis[:,:,1], **feature_params)
                
        cv2.imshow('Object Mask Tracking', output)

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import numpy as np

# Create a 2D array
my_array = np.array([[1, 2, 3], [4, 5, 6]])

# Flatten the array using ravel
flattened_array = np.ravel(my_array)

print(my_array)
print(flattened_array)

In [35]:
cv.imshow('m',mask)
cv.waitKey(0)
cv.destroyAllWindows()

In [6]:
import cv2
import numpy as np



def optical_flow(video_path,mask_path):
    # === PARAMETERS ===
    feature_params = dict(maxCorners=4, qualityLevel=0.3, minDistance=7, blockSize=7)
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
        # === INITIALIZATION ===
    cap = cv2.VideoCapture(video_path)
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    
    ## output initialization
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter('optical_flow_output.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'),  # or use 'XVID' for .avi
                      fps, (frame_width, frame_height))
    
    # Mask for features only inside the initial bounding box
    mask = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
    # Detect initial features to track
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=mask, **feature_params)
    mask_lines = np.zeros_like(old_frame)
    while True:
        ret, frame = cap.read()
        if not ret:
            out.release()
            print("breaking")
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]
            print("In")
            # Draw the tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                cv2.line(mask_lines, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
                cv2.circle(frame, (int(a), int(b)), 3, (0, 0, 255), -1)

            # Update previous frame and points
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

        # Overlay lines on the current frame
        output = cv2.add(frame, mask_lines)
        out.write(output)
        # cv2.imshow('Optical Flow Tracking', output)

        if cv2.waitKey(30) & 0xFF == 27:  # Press ESC to break
            break
        
        cap.release()
        cv2.destroyAllWindows()


In [ ]:
mask = 'YCBvideo/data/0001/000001-depth.png'
optical_flow('sample1.mp4',mask)
original_img = 'YCBvideo/data/0001/000001-color.jpg'

In
breaking


In [ ]:
m = cv.imread(mask)
m[m>0] = 255
cv.imshow('m',m)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
import numpy as np

def modify_array(arr):
  """
  Modifies a 3D numpy array:
  - Changes values greater than 0 to 0.
  - Changes values equal to 0 to 255.

  Args:
      arr: The input 3D numpy array.

  Returns:
      The modified numpy array.
  """
  modified_arr = np.where(arr > 0, 0, arr)
  modified_arr = np.where(modified_arr == 0, 255, modified_arr)
  return modified_arr

# Example usage:
# Assuming you have a 3D numpy array named 'my_array'
my_array = np.array([[[1, 2, 0], [0, 3, 4]], [[5, 0, 1], [2, 0, 0]]])
modified_array = modify_array(my_array)
print(modified_array)


[[[255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]]]
